In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings('ignore', category = DeprecationWarning)
warnings.filterwarnings('ignore', category = FutureWarning)

In [ ]:
import os
os.getcwd()

In [ ]:
#/kaggle/input/datahub/data/husky/BWYOINPD0KB6.jpg
os.chdir('/kaggle/input/datahub/')

In [ ]:
os.listdir()

#### Load data

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
data_dir = 'data'
batch_size = 256

data = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir, 
    shuffle = True,
    batch_size = batch_size)

In [ ]:
class_names = data.class_names; print('Class labels of the data:', class_names)

#### Vizuals

In [ ]:
def data_splitter(dataset, train_size, validate_size):
    '''
    A custom that splits given dataset into 
    '''
    n = len(dataset)
    train_size = round(train_size * len(dataset))
    validate_size = round(validate_size * len(data))

    train_data = dataset.take(train_size) # training dataset
    test_data = data.skip(train_size) # testing dataset
    validate_data = dataset.take(validate_size) # validation dataset
    
    return train_data, test_data, validate_data

data_list = data_splitter(data, 0.7, 0.1)
train_data = data_list[0]
test_data = data_list[1]
validate_data = data_list[1]

#### Preprocessing

In [ ]:
from tensorflow.keras import models, layers, Sequential

In [ ]:
img_size = 256
factor = (-0.3, 0.3)

resize_and_normalize = Sequential([
    layers.Resizing(img_size, img_size),
    layers.Rescaling(1/255)
    ])

augmentation = Sequential([
    layers.RandomContrast(factor = 0.5),
    layers.RandomBrightness(factor),
    layers.RandomFlip(),
    layers.RandomRotation(factor = factor),
    layers.RandomTranslation(height_factor = factor,
                             width_factor = factor),
    layers.RandomZoom(height_factor = factor,
                      width_factor = factor)
    ])

#### Define model

In [ ]:
num_classes = len(class_names)

model = Sequential([
  resize_and_normalize,
  layers.Conv2D(16, 3, padding = 'same', activation = 'relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation = 'relu')
])

In [ ]:
model.compile(optimizer='adam',
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics = ['accuracy'])

In [ ]:
history = model.fit(
    train_data,
    validation_data=validate_data,
    epochs = 10,
)

In [ ]:
weights = model.get_weights()

In [ ]:
weights

In [ ]:
weights_dir = '/kaggle/working/'  # Replace with your desired directory path
os.makedirs(weights_dir, exist_ok=True)

weights_file_path = os.path.join(weights_dir, 'model_weights1.npy')
weights = model.get_weights()
np.save(weights_file_path, weights)

In [ ]:
scores = model.evaluate(test_data)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
EPOCHS = 10
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(range(EPOCHS), acc, label='Training Accuracy')
plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), loss, label='Training Loss')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()